In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import multiprocessing
import os
import numpy as np
import cv2
import shutil
import pickle
from google.colab.patches import cv2_imshow
import difflib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Train directory
os.chdir("/content/drive/My Drive/Colab Notebooks/train")
images_directory = "/content/drive/My Drive/Colab Notebooks/tr"

# Results directory
result_directory = "/content/drive/My Drive/Colab Notebooks/results"

# Model path
model_path = result_directory + "/models.pickle"

# Pictures to check
check_path = "/content/drive/My Drive/Colab Notebooks/ch"


# Free if exists
if os.path.isdir(result_directory):
  shutil.rmtree(result_directory)

# Create directory and change to
os.mkdir(result_directory)
os.chdir(result_directory)


print("Текущая деректория:", os.getcwd())

Текущая деректория: /content/drive/My Drive/Colab Notebooks/results


In [ ]:
def find_start(image_map):
  for i in range(image_to_map_size):
    for j in reversed(range(image_to_map_size)):
      if (image_map[:,i][j] != 0):
        return j, i

def dir_1(image_map, i, j):
  if (i == 0): return False
  if (image_map[i - 1][j] == 1):
    return True
  else:
    return False

def dir_2(image_map, i, j):
  if (i == 0 or j == image_to_map_size - 1): return False
  if (image_map[i - 1][j + 1] == 1):
    return True
  else:
    return False

def dir_3(image_map, i, j):
  if (j == image_to_map_size - 1): return False
  if (image_map[i][j + 1] == 1):
    return True
  else:
    return False

def dir_4(image_map, i, j):
  if (i == image_to_map_size - 1 or j == image_to_map_size - 1): return False
  if (image_map[i + 1][j + 1] == 1):
    return True
  else:
    return False

def dir_5(image_map, i, j):
  if (i == image_to_map_size - 1): return False
  if (image_map[i + 1][j] == 1):
    return True
  else:
    return False

def dir_6(image_map, i, j):
  if (i == image_to_map_size - 1 or j == 0): return False
  if (image_map[i + 1][j - 1] == 1):
    return True
  else:
    return False

def dir_7(image_map, i, j):
  if (j == 0): return False
  if (image_map[i][j - 1] == 1):
    return True
  else:
    return False
  
def dir_8(image_map, i, j):
  if (i == 0 or j == 0): return False
  if (image_map[i - 1][j - 1] == 1):
    return True
  else:
    return False

def neighbour(image_map, i, j):
  neighbour_num = 0
  direction = 0
  i_n = -1
  j_n = -1
  found = False
  if (dir_1(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 1
      i_n = i - 1
      j_n = j
  if (dir_2(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 2
      i_n = i - 1
      j_n = j + 1
  if (dir_3(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 3
      i_n = i
      j_n = j + 1
  if (dir_4(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 4
      i_n = i + 1
      j_n = j + 1
  if (dir_5(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 5
      i_n = i + 1
      j_n = j
  if (dir_6(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 6
      i_n = i + 1
      j_n = j - 1
  if (dir_7(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 7
      i_n = i
      j_n = j - 1
  if (dir_8(image_map, i, j)):
    neighbour_num += 1
    if (found == False):
      found = True
      direction = 8
      i_n = i - 1
      j_n = j - 1

  return i_n, j_n, direction, neighbour_num

In [ ]:
def get_syntax(st_bypass):
  st_bypass = st_bypass.replace("(00", "")
  st_bypass = st_bypass.replace("(0)", "")
  st_bypass = st_bypass.replace("0", "")
  for i in range(10):
    st_bypass = st_bypass.replace("11", "1")
    st_bypass = st_bypass.replace("22", "2")
    st_bypass = st_bypass.replace("33", "3")
    st_bypass = st_bypass.replace("44", "4")
    st_bypass = st_bypass.replace("55", "5")
    st_bypass = st_bypass.replace("66", "6")
    st_bypass = st_bypass.replace("77", "7")
    st_bypass = st_bypass.replace("88", "8")

  for i in range(10):
    st_bypass = st_bypass.replace("(", "+(")
    st_bypass = st_bypass.replace("1", "+a")
    st_bypass = st_bypass.replace("2", "+c")
    st_bypass = st_bypass.replace("3", "+b")
    st_bypass = st_bypass.replace("4", "+d")
    st_bypass = st_bypass.replace("5", "-a")
    st_bypass = st_bypass.replace("6", "-c")
    st_bypass = st_bypass.replace("7", "-d")
    st_bypass = st_bypass.replace("8", "-d")
    st_bypass = st_bypass.replace("(+", "(")
    
  for i in range(5):
    st_bypass = st_bypass.replace("++", "+")
  return st_bypass

In [ ]:
def main_bypass(image_map_):
  i_s, j_s = find_start(image_map_)
  # Main Cycle
  st_bypass = ""
  i_head, j_head = -1, -1
  i_cur, j_cur = i_s, j_s
  i_next, j_next = i_s, j_s
  i_oth, j_oth = -1, -1
  direction = -1
  neighbour_num = -1
  neighbour_list = []

  while True:
    i_cur, j_cur = i_next, j_next
    if (image_map_[i_cur][j_cur] == 1):
      image_map_[i_cur][j_cur] = 2
    i_next, j_next, direction, neighbour_num = neighbour(image_map_, i_cur, j_cur)
    if (neighbour_num > 1):
      i_put, j_put = i_cur, j_cur

      if (i_head != -1 and j_head != -1):
        st_bypass += ')'

        i_next, j_next = i_head, j_head
          

        i_oth, j_oth, direction_oth, neighbour_num_oth = neighbour(image_map_, i_next, j_next)
        if (neighbour_num_oth == 1):
          image_map_[i_next][j_next] = 3
          if (len(neighbour_list) > 0):
            i_head, j_head = neighbour_list[0]   
            del neighbour_list[0]
          else:
            i_head, j_head = -1, -1
        else:
          i_next, j_next, direction, neighbour_num = neighbour(image_map_, i_cur, j_cur)
        if (i_head == -1 and j_head == -1):
          i_phead, j_phead = i_head, j_head
          i_head, j_head = i_put, j_put
        else:
          neighbour_list.append((i_put, j_put))
            
        image_map_[i_cur][j_cur] = 4
      else:
        i_head, j_head = i_put, j_put
        image_map_[i_cur][j_cur] = 4
      st_bypass += '('
    if (i_cur == i_next and j_cur == j_next and i_head != -1 and j_head != -1):
      i_next, j_next = i_head, j_head
      st_bypass += ")("

    st_bypass += str(direction)
    if ("00" in st_bypass):
      break
  return get_syntax(st_bypass)

In [ ]:
def create_model(path, o, images_directory, return_dict):
  images_path = images_directory + "/" + path
  count = 0
  for image_path in os.listdir(images_path):
    count += 1
    full_path = images_path + "/" + image_path
    image = cv2.imread(full_path)
    image_map = [[0] * 16 for i in range(16)]
    for i in range(len(image)):
      for j in range(len(image[0])):
        if (image[i][j][0] != 255):
          image_map[i][j] = 1
        else:
          image_map[i][j] = 0
  image_map = np.array(image_map)
  syntax = main_bypass(image_map)
  return_dict[o] = (path, syntax)

def create_check_model(path):
  full_path = check_path + '/' + path
  image = cv2.imread(full_path)
  image_map = [[0] * 16 for i in range(16)]
  for i in range(len(image)):
    for j in range(len(image[0])):
      if (image[i][j][0] != 255):
        image_map[i][j] = 1
      else:
        image_map[i][j] = 0
  image_map = np.array(image_map)
  syntax = main_bypass(image_map)  
  return syntax
  #???

  # return image_map

In [ ]:
# Main Cycle
procs = []
lock = multiprocessing.Lock()

image_to_map_size = 16

manager = multiprocessing.Manager()
return_dict = manager.dict()
for j, i in enumerate(os.listdir(images_directory)):
  create_model(i, j, images_directory, return_dict)
  p = multiprocessing.Process(target=create_model, args=(i, j, images_directory, return_dict, ))
  procs.append(p)
  p.start()

[proc.join() for proc in procs]


models = sorted(return_dict.values())
if (os.path.isfile(model_path)):
    os.remove(model_path)

with open(model_path, 'wb') as f:
  pickle.dump(models, f)



In [ ]:
upload_models = []
with open(model_path, 'rb') as f:
     upload_models = pickle.load(f)


In [ ]:
check_models = []

for image_path in os.listdir(check_path):
  check_models.append(create_check_model(image_path))


for i, check_model in enumerate(check_models):
  closest = ("$", -1)
  for tuple_ in upload_models:
    ans = tuple_[0]
    model = tuple_[1]
    sim = (difflib.SequenceMatcher(None, check_model, model)).ratio()
    if (sim > closest[1]):
      closest = (ans, sim)
  image = cv2.imread(check_path + '/' + os.listdir(check_path)[i])
  print(closest[0])
  cv2_imshow((image))

8


0


7


In [ ]:
ii = 3
pic = sorted(os.listdir(images_directory + "/" + sorted(os.listdir(images_directory))[ii]))[0]
path = images_directory + "/" + sorted(os.listdir(images_directory))[ii]
pic_path = path + "/" + pic
image = cv2.imread(path + "/" + pic)
print(pic_path)
image_map = [[0] * 16 for i in range(16)]
for i in range(len(image)):
  for j in range(len(image[0])):
    if (image[i][j][0] != 255):
      image_map[i][j] = 1
    else:
      image_map[i][j] = 0



image_map = np.array(image_map)

print(main_bypass(image_map))
savemap = image_map
image_to_map_size = 16

In [ ]:
i_s, j_s = find_start(image_map)
# Main Cycle
st_bypass = ""
i_head, j_head = -1, -1
i_cur, j_cur = i_s, j_s
i_next, j_next = i_s, j_s
i_oth, j_oth = -1, -1
direction = -1
neighbour_num = -1
neighbour_list = []

while True:
  i_cur, j_cur = i_next, j_next
  if (image_map[i_cur][j_cur] == 1):
    image_map[i_cur][j_cur] = 2
  i_next, j_next, direction, neighbour_num = neighbour(image_map, i_cur, j_cur)
  if (neighbour_num > 1):
    i_put, j_put = i_cur, j_cur

    if (i_head != -1 and j_head != -1):
      st_bypass += ')'

      i_next, j_next = i_head, j_head
        

      i_oth, j_oth, direction_oth, neighbour_num_oth = neighbour(image_map, i_next, j_next)
      if (neighbour_num_oth == 1):
        image_map[i_next][j_next] = 3
        if (len(neighbour_list) > 0):
          i_head, j_head = neighbour_list[0]   
          del neighbour_list[0]
        else:
          i_head, j_head = -1, -1
      else:
        i_next, j_next, direction, neighbour_num = neighbour(image_map, i_cur, j_cur)
      if (i_head == -1 and j_head == -1):
        i_phead, j_phead = i_head, j_head
        i_head, j_head = i_put, j_put
      else:
        neighbour_list.append((i_put, j_put))
          
      image_map[i_cur][j_cur] = 4
    else:
      i_head, j_head = i_put, j_put
      image_map[i_cur][j_cur] = 4
    st_bypass += '('
  if (i_cur == i_next and j_cur == j_next and i_head != -1 and j_head != -1):
    i_next, j_next = i_head, j_head
    st_bypass += ")("

  st_bypass += str(direction)
  if ("00" in st_bypass):
    break
print(image_map)
print(st_bypass)